In [1]:
import h2o
from h2o.automl import H2OAutoML
from h2o.estimators import H2OWord2vecEstimator

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,1 hour 38 mins
H2O_cluster_timezone:,Asia/Colombo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.1
H2O_cluster_version_age:,"21 days, 5 hours and 29 minutes"
H2O_cluster_name:,H2O_from_python_mathanraj_5vin33
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.786 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [3]:
# Import the craigslist dataset into H2O:
job_titles = h2o.import_file(
    ("https://s3.amazonaws.com/h2o-public-test-data/smalldata/craigslistJobTitles.csv"),
    col_names = ["category", "jobtitle"],
    col_types = ["string", "string"],
    header = 1
)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [4]:
STOP_WORDS = [
    "ax","i","you","edu","s","t","m","subject","can",
    "lines","re","what","there","all","we","one","the",
    "a","an","of","or","in","for","by","on","but","is",
    "in","a","not","with","as","was","if","they","are",
    "this","and","it","have","from","at","my","be","by",
    "not","that","to","from","com","org","like","likes",
    "so"
]

In [5]:
# Make the 'tokenize' function:
def tokenize(sentences, stop_word = STOP_WORDS):
    tokenized = sentences.tokenize("\\W+")
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
    return tokenized_words

In [6]:
# Make the `predict` function:
def predict(job_title, w2v, gbm):
    words = tokenize(h2o.H2OFrame(job_title).ascharacter())
    job_title_vec = w2v.transform(words, aggregate_method="AVERAGE")
    print(gbm.predict(test_data=job_title_vec))

In [7]:
# Break job titles into a sequence of words:
words = tokenize(job_titles["jobtitle"])

# Build word2vec model:
w2v_model = H2OWord2vecEstimator(sent_sample_rate = 0.0, epochs = 10)
w2v_model.train(training_frame=words)

# Find synonyms for the words "teacher":
w2v_model.find_synonyms("teacher", count = 5)

word2vec Model Build progress: |█████████████████████████████████████████████████| (done) 100%


OrderedDict([('teaching', 0.6702170372009277),
             ('infant', 0.6643931269645691),
             ('preschool', 0.6431465148925781),
             ('aide', 0.6305525302886963),
             ('toddler', 0.6170276403427124)])

In [8]:
# Calculate a vector for each job title:
job_title_vecs = w2v_model.transform(words, aggregate_method = "AVERAGE")

# Prepare training & validation data (keep only job titles made of known words):
valid_job_titles = ~ job_title_vecs["C1"].isna()
job_titles["category"] = job_titles["category"].asfactor()
data = job_titles[valid_job_titles,:].cbind(job_title_vecs[valid_job_titles,:])

train, valid, test = data.split_frame(ratios=[0.7, 0.2])

In [10]:
# Build a AutoML model:
aml = H2OAutoML(
    nfolds=5,
    max_models=5,
    max_runtime_secs=60, 
    seed=1234
)

aml.train(x=job_title_vecs.names ,y="category", training_frame=train, validation_frame=valid)

AutoML progress: |
23:19:56.465: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.
23:19:56.466: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGeneralizedLinearEstimator : Generalized Linear Modeling
Model Key: GLM_1_AutoML_2_20230711_231956


GLM Model: summary
    family       link         regularization               lambda_search                                                                   number_of_predictors_total    number_of_active_predictors    number_of_iterations    training_frame
--  -----------  -----------  ---------------------------  ------------------------------------------------------------------------------  ----------------------------  -----------------------------  ----------------------  ------------------------------------------------
    multinomial  multinomial  Ridge ( lambda = 5.606E-4 )  nlambda = 30, lambda.max = 19.97, lambda.min = 5.606E-4, lambda.1se = 0.006069  606                           600                            59                      AutoML_2_20230711_231956_training_py_71_sid_b6ba

ModelMetricsMultinomialGLM: glm
** Reported on train data. **

MSE: 0.19126263562219772
RMSE: 0.4373358384836506
LogLoss: 0.6090590414073777
Null degrees of freedom: 9650
Residual degrees of freedom: 9045
Null deviance: 34366.511703697164
Residual deviance: 11756.057617245191
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
accounting    administrative    customerservice    education    foodbeverage    labor    Error      Rate
------------  ----------------  -----------------  -----------  --------------  -------  ---------  -------------
891           115               55                 17           16              11       0.193665   214 / 1,105
88            1386              184                36           30              36       0.2125     374 / 1,760
27            178               993                43           128             235      0.380923   611 / 1,604
10            41                36                 1576         15              17       0.0702065  119 / 1,695
2             46                117                12           1426            157      0.189773   334 / 1,760
13            31                159                31           117             1376     0.203243   351 / 1,727
1031          1797              1544               1715         1732            1832     0.207543   2,003 / 9,651

Top-6 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.792457
2    0.914102
3    0.968604
4    0.984872
5    0.995026
6    1

ModelMetricsMultinomialGLM: glm
** Reported on validation data. **

MSE: 0.19708105109837445
RMSE: 0.4439381162936727
LogLoss: 0.6430418691346254
Null degrees of freedom: 2704
Residual degrees of freedom: 2099
Null deviance: 9639.724697801532
Residual deviance: 3478.8565120183166
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
accounting    administrative    customerservice    education    foodbeverage    labor    Error     Rate
------------  ----------------  -----------------  -----------  --------------  -------  --------  -----------
271           23                13                 6            2               2        0.14511   46 / 317
20            359               58                 14           9               16       0.245798  117 / 476
10            50                273                16           38              66       0.397351  180 / 453
3            

In [13]:
aml.leaderboard

model_id,mean_per_class_error,logloss,rmse,mse
GLM_1_AutoML_2_20230711_231956,0.223878,0.664172,0.450748,0.203173


In [16]:
best_model = aml.get_best_model()

In [17]:
# Predict
print(predict(["school teacher having holidays every month"], w2v_model, best_model))
print(predict(["developer with 3+ Java experience, jumping"], w2v_model, best_model))
print(predict(["Financial accountant CPA preferred"], w2v_model, best_model))

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
predict      accounting    administrative    customerservice    education    foodbeverage      labor
education    0.00103003         0.0459678          0.0214857     0.899527     0.000945136  0.0310447
[1 row x 7 columns]

None
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
predict      accounting    administrative    customerservice    education    foodbeverage     labor
labor         0.0135229          0.306551           0.202388   0.00520171       0.0911395  0.381197
[1 row x 7 columns]

None
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |█████████████████████████████████████████████

In [18]:
best_model.explain()

ImportError: Plotting functionality requires matplotlib. Please install matplotlib.